In [1]:
from transformers import pipeline

import os
import sys
sys.path.append("..")
import pandas as pd

2024-04-17 14:35:55.683047: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 14:35:56.865963: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      return_all_scores=False)

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/ucloud/.local/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
filename = os.path.join("..", "in", "Game_of_Thrones_Script.csv")
data = pd.read_csv(filename)

In [24]:
data

,Release Date,Season,Episode,Episode Title,Name,Sentence
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.
...,...,...,...,...,...,...
23906,2019-05-19,Season 8,Episode 6,The Iron Throne,brienne,I think we can all agree that ships take prece...
23907,2019-05-19,Season 8,Episode 6,The Iron Throne,bronn,I think that's a very presumptuous statement.
23908,2019-05-19,Season 8,Episode 6,The Iron Throne,tyrion lannister,I once brought a jackass and a honeycomb into ...
23909,2019-05-19,Season 8,Episode 6,The Iron Throne,man,The Queen in the North!


In [7]:
simple_data = data.loc[:, ["Season", "Sentence"]]

In [18]:
print(simple_data)

         Season                                           Sentence
0      Season 1  What do you expect? They're savages. One lot s...
1      Season 1  I've never seen wildlings do a thing like this...
2      Season 1                             How close did you get?
3      Season 1                            Close as any man would.
4      Season 1                   We should head back to the wall.
...         ...                                                ...
23906  Season 8  I think we can all agree that ships take prece...
23907  Season 8      I think that's a very presumptuous statement.
23908  Season 8  I once brought a jackass and a honeycomb into ...
23909  Season 8                            The Queen in the North!
23910  Season 8  The Queen in the North! The Queen in the North...

[23911 rows x 2 columns]


In [51]:
labels = []
for line in data["Sentence"]:
    if isinstance(line, str): 
        line = [line]  # Convert to a list of strings
    elif not isinstance(line, list):
        print("Warning: Line is not a string or list, skipping it.")
        continue
        
    try:
        label = classifier(line)
        labels.append(label[0]["label"])
    except Exception as e:
        print(f"Error processing line: {line}")
        print(e)
        data.drop(index, inplace=True)
        continue


In [53]:
print(labels)

['disgust', 'disgust', 'surprise', 'neutral', 'fear', 'fear', 'neutral', 'surprise', 'anger', 'anger', 'neutral', 'neutral', 'neutral', 'surprise', 'neutral', 'neutral', 'anger', 'neutral', 'neutral', 'joy', 'neutral', 'neutral', 'neutral', 'surprise', 'disgust', 'neutral', 'surprise', 'disgust', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'sadness', 'neutral', 'anger', 'neutral', 'fear', 'neutral', 'neutral', 'sadness', 'neutral', 'surprise', 'disgust', 'anger', 'neutral', 'anger', 'surprise', 'neutral', 'surprise', 'anger', 'surprise', 'neutral', 'neutral', 'disgust', 'disgust', 'disgust', 'neutral', 'neutral', 'anger', 'sadness', 'neutral', 'neutral', 'neutral', 'anger', 'anger', 'anger', 'anger', 'sadness', 'neutral', 'anger', 'surprise', 'anger', 'surprise', 'neutral', 'fear', 'fear', 'fear', 'anger', 'neutral', 'sadness', 'anger', 'disgust', 'neutral', 'neutral', 'neutral', 'surprise', 'neutral', 'sadness', 'neutral', 'sadness', 'neutral', 'neutral', 'neutra

In [54]:
data["labels"] = labels

ValueError: Length of values (23910) does not match length of index (23911)

In [26]:
test

,Season,Sentence,labels
0,Season 1,What do you expect? They're savages. One lot s...,disgust
1,Season 1,I've never seen wildlings do a thing like this...,disgust
2,Season 1,How close did you get?,surprise
3,Season 1,Close as any man would.,neutral
4,Season 1,We should head back to the wall.,fear
5,Season 1,Do the dead frighten you?,fear
6,Season 1,Our orders were to track the wildlings. We tra...,neutral
7,Season 1,You don't think he'll ask us how they died? Ge...,surprise
8,Season 1,Whatever did it to them could do it to us. The...,anger
9,Season 1,It's a good thing we're not children. You want...,anger
